In [1]:
from ERH.elements.Planetary import Planetary
from ERH.eq.conversion import *

EngranePlanetario=Planetary()

In [2]:
#(n_c - n_pp) / (n_pp - n_s) = N_s / N_c       #Relacion de velocidades
#(n_p - n_pp) / (n_pp - n_s) = N_s / N_p
#N_c = N_s + 2*N_p

In [3]:
EngranePlanetario.create_planetary_sistem(m=1,phi_n=20,F=5,N_c=96,N_s=24,N_p=36,ctd_planetas=3)

In [4]:
EngranePlanetario.get_np_from(npp=200,ns=1000)

Resultado para la velocidad angular del planeta -333.33



In [5]:
H = 150
EngranePlanetario.set_load_corona(w=rpm_a_rad_s(0),h=H,t=None)
EngranePlanetario.set_load_planeta(w=rpm_a_rad_s(333.333),h=H,t=None)
EngranePlanetario.set_load_sol(w=rpm_a_rad_s(1000),h=H,t=None)

rpm_a_rad_s(0) = 0.0 rad/s
rpm_a_rad_s(333.333) = 34.91 rad/s
rpm_a_rad_s(1000) = 104.72 rad/s


In [6]:
EngranePlanetario.set_Wt()
EngranePlanetario.set_ciclos(horas=24,dias=365,years=1)

Esfuerzo del sol tangencial 119.36592818945762
Esfuerzo percibido por los elementos 39.79

=== Ciclos acumulados ===
Sol     : 2.10e+08 ciclos, para calculo 6.31e+08 
Planeta : -1.40e+08 ciclos, para calculo -1.40e+08
Corona  : -3.15e+08 ciclos, para calculo -9.46e+08



In [7]:
K_A = 1        # PAG 32 TABLA 17: factor de aplicación
K_B = 1        # PAG 32 (abajo) TABLA 17: factor de tamaño
K_M = 1.6      # PAG 32 TABLA 18: factor de montaje
K_S = 1        # PAG 33 TABLA 20: factor de seguridad
C_F = 1        # PAG 31: 1 normal; >1 si superficie áspera
Q_v = 10       # Calidad del engrane
C_p = 229.06   # PAG 31 TABLA 15: coef. elasticidad [√MPa]
I   = 0.07     # Índice geométrico de contacto

tension_sol_planeta = {
    "K_A": K_A, "K_M": K_M, "K_S": K_S, "K_B": K_B, "K_V": None,   # Factores AGMA (páginas/tablas)
    "K_I_pin": 1.00, "K_I_eng": 1.42,                              # Factor de ubicación piñón/engranaje
    "Jp_pin": 0.35, "Jmod_pin": 1,                                 # Geometría piñón (gráficos 27–29)
    "Jp_eng": 0.38, "Jmod_eng": 1,                                 # Geometría engrane (gráficos 27–29)
    "C_p": C_p, "C_F": C_F, "Q_v": Q_v,                            # Contacto, acabado, calidad
    "I": I, "Jp": None, "Jg": None,                                # Índice geométrico y opcionales
}


tension_planeta_corona = {
    "K_A": K_A, "K_M": K_M, "K_S": K_S, "K_B": K_B, "K_V": None,
    "K_I_pin": 1.42, "K_I_eng": 1.00,
    "Jp_pin": tension_sol_planeta.get("Jp_eng"),
    "Jmod_pin": tension_sol_planeta.get("Jmod_eng"),
    "Jp_eng": 0.44, "Jmod_eng": 1,
    "C_p": C_p, "C_F": C_F, "Q_v": Q_v,
    "I": I, "Jp": None, "Jg": None,
}


In [8]:
EngranePlanetario.calc_esfuerzos(tension_sol_planeta, tension_planeta_corona)

Se introdujo I:  0.07


In [9]:
Temp = 20      # °C
R    = 0.99    # Confiabilidad objetivo (0–1)

resistance_sol_planeta = {
    "K_L_p": 0.85, "C_L_p": 0.75, "HB_p": 235,       # Piñón
    "K_L_g": 0.91, "C_L_g": 0.85, "HB_g": 235,       # Engrane
    "temperatura": Temp, "R": R,                     # Servicio (°C, confiabilidad)
    "caso_engrane": "masa", "Rq": 0.8,               # Caso: masa/superficie, rugosidad (µm)
    "pSF_g": 210.26, "pSFC_g": 707.75,               # Engrane permisible (flexión/contacto)
    "pSF_p": 210.26, "pSFC_p": 707.75,               # Piñón permisible (flexión/contacto)
    "K_T": None, "K_R": None, "C_H_p": None, "C_H_g": None,   # Factores opcionales
}

resistance_planeta_corona = {
    "K_L_p": resistance_sol_planeta.get("K_L_g"),
    "C_L_p": resistance_sol_planeta.get("C_L_g"),
    "HB_p":  resistance_sol_planeta.get("HB_g"),
    "K_L_g": 1, "C_L_g": 1, "HB_g": 300,
    "temperatura": Temp, "R": R,
    "caso_engrane": "masa", "Rq": 0.8,
    "pSF_g": 324, "pSFC_g": 942,
    "pSF_p": resistance_sol_planeta.get("pSF_g"),
    "pSFC_p": resistance_sol_planeta.get("pSFC_g"),
    "K_T": None, "K_R": None, "C_H_p": None, "C_H_g": None,
}


In [10]:
EngranePlanetario.calc_resistencia(resistance_sol_planeta,resistance_planeta_corona)